In [24]:
import re
import pandas as pd
import numpy as np
from numpy import dtype
pd.set_option('display.max_columns', None)

In [32]:
def int_strip(text):
    val = text.lower()
    try:
        return int(val)
    except ValueError:
        print('"%s"' % text)
        return None
    
def int_float_strip(text):
    val = text.lower()
    try:
        return int(float(val))
    except ValueError:
        print('"%s"' % text)
        return None

In [40]:
na_values=['NA']
nrows = 1000
corrupted = pd.read_csv('train.csv.zip', na_values = na_values, skipinitialspace = True, 
                        parse_dates=['fecha_dato', 'fecha_alta'], nrows=nrows)

In [41]:
def month(date):
    return date.month

def year(date):
    return date.year

corrupted = corrupted[corrupted.age.notnull()]
corrupted = corrupted[corrupted.ind_nomina_ult1.notnull()]
corrupted = corrupted[corrupted.cod_prov.notnull()]
corrupted = corrupted[corrupted.ind_nom_pens_ult1.notnull()]
corrupted = corrupted.drop('nomprov', 1)

m = corrupted.fecha_dato.apply(month)
y = corrupted.fecha_dato.apply(year)

corrupted.insert(2, 'fecha_dato_m', m)
corrupted.insert(2, 'fecha_dato_y', y)
corrupted = corrupted.drop('fecha_dato', 1)

m = corrupted.fecha_alta.apply(month)
y = corrupted.fecha_alta.apply(year)

corrupted.insert(7, 'fecha_alta_m', m)
corrupted.insert(7, 'fecha_alta_y', y)
corrupted = corrupted.drop('fecha_alta', 1)

train = corrupted
train.head()

,fecha_dato_y,fecha_dato_m,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta_y,fecha_alta_m,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015,1,1375586,N,ES,H,35.0,2015,1,0.0,6.0,1.0,NaN,1.0,A,S,N,NaN,KHL,N,1.0,29.0,1.0,87218.10,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2015,1,1050611,N,ES,V,23.0,2012,8,0.0,35.0,1.0,NaN,1.0,I,S,S,NaN,KHE,N,1.0,13.0,0.0,35548.74,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2015,1,1050612,N,ES,V,23.0,2012,8,0.0,35.0,1.0,NaN,1.0,I,S,N,NaN,KHE,N,1.0,13.0,0.0,122179.11,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2015,1,1050613,N,ES,H,22.0,2012,8,0.0,35.0,1.0,NaN,1.0,I,S,N,NaN,KHD,N,1.0,50.0,0.0,119775.54,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2015,1,1050614,N,ES,V,23.0,2012,8,0.0,35.0,1.0,NaN,1.0,A,S,N,NaN,KHE,N,1.0,50.0,1.0,NaN,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [42]:
def sub_code_and_toint(text):
    return float(str(text)[1:2])
    

def preprocess_features(X):
    ''' Preprocess string data to one-hot data such as state name, timestamps and so on '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)
    i = 0
    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        if col == 'ind_empleado':
            col_data = col_data.replace([1.0, 99.0], [1.0, 0.0])
        
        if col in ['indext', 'indresi']:
            col_data = col_data.replace(['S', 'N'], [1, 0])
        
        if col == 'sexo':
            col_data = col_data.replace(['V', 'H'], [1.0, 0.0])
        
        if col == 'segmento':
            col_data = col_data.apply(sub_code_and_toint)
        
        # If data type is categorical, convert to dummy variables 
        if col in ['ind_empleado', 'pais_residencia', 'tiprel_1mes', 'canal_entrada']:
            col_data = pd.get_dummies(col_data, prefix = col) 
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

def process_dates(X):
    fecha_dato = X.fecha_dato
    fecha_alta = X.fecha_alta
    
    

In [43]:
train = preprocess_features(train)

In [44]:
train.head()

,fecha_dato_y,fecha_dato_m,ncodpers,ind_empleado_N,pais_residencia_ES,sexo,age,fecha_alta_y,fecha_alta_m,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes_A,tiprel_1mes_I,indresi,indext,conyuemp,canal_entrada_KAT,canal_entrada_KAZ,canal_entrada_KFA,canal_entrada_KFC,canal_entrada_KHD,canal_entrada_KHE,canal_entrada_KHL,canal_entrada_RED,indfall,tipodom,cod_prov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015,1,1375586,1.0,1.0,0.0,35.0,2015,1,0.0,6.0,1.0,NaN,1.0,1.0,0.0,1,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,N,1.0,29.0,1.0,87218.10,2.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2015,1,1050611,1.0,1.0,1.0,23.0,2012,8,0.0,35.0,1.0,NaN,1.0,0.0,1.0,1,1,NaN,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,N,1.0,13.0,0.0,35548.74,3.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2015,1,1050612,1.0,1.0,1.0,23.0,2012,8,0.0,35.0,1.0,NaN,1.0,0.0,1.0,1,0,NaN,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,N,1.0,13.0,0.0,122179.11,3.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2015,1,1050613,1.0,1.0,0.0,22.0,2012,8,0.0,35.0,1.0,NaN,1.0,0.0,1.0,1,0,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,N,1.0,50.0,0.0,119775.54,3.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2015,1,1050614,1.0,1.0,1.0,23.0,2012,8,0.0,35.0,1.0,NaN,1.0,1.0,0.0,1,0,NaN,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,N,1.0,50.0,1.0,NaN,3.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
